## Project 3: Digit Recognition

Good programmers can use neural nets. Great programmers can make them. This section will guide you through the implementation of a simple neural net with an architecture as shown in the figure below. You will implement the net from scratch (you will probably never do this again, don't worry) so that you later feel confident about using libraries. We provide some skeleton code in neural_nets.py for you to fill in.

![neural_net](../Media/images_neuralnet.png)

In [102]:
import numpy as np
import matplotlib.pyplot as plt
import torch

### 3. Activation Functions

#### Rectified Linear Unit

In [3]:
def rectified_linear_unit(x):
    """ Returns the ReLU of x, or the maximum between 0 and x."""
    return np.maximum(x, 0)

#### Taking the Derivative

In [62]:
def rectified_linear_unit_derivative(x):
    """ Returns the derivative of ReLU."""

    x[x > 0] = 1
    x[x <= 0] = 0

    # ReLu returns 1 for all positive values and 0 for all negative values
    # (Returns 0 when the value is equal to 0 as well)
    return x

------
### 4. Training the Network

In [135]:
def output_layer_activation(x):
    """ Linear function, returns input as is. """
    return x

def output_layer_activation_derivative(x):
    """ Returns the derivative of a linear function: 1. """
    return 1


class NeuralNetwork():
    """
        Contains the following functions:
            -train: tunes parameters of the neural network based on error obtained from forward propagation.
            -predict: predicts the label of a feature vector based on the class's parameters.
            -train_neural_network: trains a neural network over all the data points for the specified number of epochs during initialization of the class.
            -test_neural_network: uses the parameters specified at the time in order to test that the neural network classifies the points given in testing_points within a margin of error.
    """

    def __init__(self):

        # DO NOT CHANGE PARAMETERS (Initialized to floats instead of ints)
        self.input_to_hidden_weights = np.matrix('1. 1.; 1. 1.; 1. 1.')  # (3,2)
        self.hidden_to_output_weights = np.matrix('1. 1. 1.')
        self.biases = np.matrix('0.; 0.; 0.')
        self.learning_rate = .001
        self.epochs_to_train = 10
        self.training_points = [((2,1), 10), ((3,3), 21), ((4,5), 32), ((6, 6), 42)]
        self.testing_points = [(1,1), (2,2), (3,3), (5,5), (10,10)]


    # ============================================================

    def train(self, x1 : float, x2: float, y):

        ### Forward propagation ###
        input_values = np.matrix([[x1],[x2]]) # 2 by 1

        # Calculate the input and activation of the hidden layer
        hidden_layer_weighted_input = np.dot(self.input_to_hidden_weights, input_values) + self.biases  # (3,2) * (2x1) = (3,1) + (3,1) = (3,1)
        hidden_layer_activation = rectified_linear_unit(hidden_layer_weighted_input)                    # (3,1)

        output = np.dot(self.hidden_to_output_weights, hidden_layer_activation)                         # (1,3) * (3,1) = (1,1)
        activated_output = output_layer_activation(output)

        ### Backpropagation ###

        # Compute gradients
        output_layer_error = (y - activated_output)                                                                                 # Derivative of cost function
        hidden_layer_error = self.hidden_to_output_weights.T * output_layer_activation_derivative(output) *  output_layer_error     # (3 by 1 matrix)

        bias_gradients = hidden_layer_error * 1                                                     # Derivative of Z with respect of the bias is 1 (Z = W*a + b) 
        hidden_to_output_weight_gradients = np.dot(output_layer_error, hidden_layer_activation.T)   # Derivative of Z with respect of the weights is the weighted input of the layer (Z' = a)
        input_to_hidden_weight_gradients = np.dot(hidden_layer_error, input_values.T)
        
        # print(bias_gradients.shape)
        # print(input_to_hidden_weight_gradients.shape)
        # print(hidden_to_output_weight_gradients.shape)
        # print("======================")

        # Use gradients to adjust weights and biases using gradient descent
        self.biases = self.biases - self.learning_rate * bias_gradients
        self.input_to_hidden_weights = self.input_to_hidden_weights - self.learning_rate * input_to_hidden_weight_gradients
        self.hidden_to_output_weights = self.hidden_to_output_weights - self.learning_rate * hidden_to_output_weight_gradients

    # ============================================================

    def predict(self, x1, x2):

        input_values = np.matrix([[x1],[x2]])
        print(input_values.shape)
        print(self.input_to_hidden_weights.shape)

        # Compute output for a single input(should be same as the forward propagation in training)
        hidden_layer_weighted_input = np.dot(self.input_to_hidden_weights, input_values) + self.biases
        hidden_layer_activation = rectified_linear_unit(hidden_layer_weighted_input)
        output = np.dot(self.hidden_to_output_weights, hidden_layer_activation)

        activated_output = output_layer_activation(output)
        return activated_output.item()

    # Run this to train your neural network once you complete the train method
    def train_neural_network(self):

        for epoch in range(self.epochs_to_train):
            for x,y in self.training_points:
                self.train(x[0], x[1], y)

    # Run this to test your neural network implementation for correctness after it is trained
    def test_neural_network(self):

        for point in self.testing_points:
            print("Point,", point, "Prediction,", self.predict(point[0], point[1]))
            if abs(self.predict(point[0], point[1]) - 7*point[0]) < 0.1:
                print("Test Passed")
            else:
                print("Point ", point[0], point[1], " failed to be predicted correctly.")
                return

In [136]:
x = NeuralNetwork()
x.train_neural_network()
x.test_neural_network()

(2, 1)
(3, 2)
Point, (1, 1) Prediction, -inf
(2, 1)
(3, 2)
Point  1 1  failed to be predicted correctly.


### 8. Fully-Connected Neural Networks

#### Training and Testing Accuracy Over Time

In [ ]:
import _pickle as cPickle, gzip
import numpy as np
from tqdm import tqdm
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import sys

sys.path.append("..")
from .mnist.utils import *
from .mnist.part2-mnist.train_utils import batchify_data, run_epoch, train_model

# Specify seed for deterministic behavior, then shuffle. Do not change seed for official submissions to edx
np.random.seed(12321)  # for reproducibility
torch.manual_seed(12321)  # for reproducibility

# Load the dataset
num_classes = 10
X_train, y_train, X_test, y_test = get_MNIST_data()

# Split into train and dev
dev_split_index = int(9 * len(X_train) / 10)
X_dev = X_train[dev_split_index:]
y_dev = y_train[dev_split_index:]
X_train = X_train[:dev_split_index]
y_train = y_train[:dev_split_index]

permutation = np.array([i for i in range(len(X_train))])
np.random.shuffle(permutation)
X_train = [X_train[i] for i in permutation]
y_train = [y_train[i] for i in permutation]

# Split dataset into batches
batch_size = 32
train_batches = batchify_data(X_train, y_train, batch_size)
dev_batches = batchify_data(X_dev, y_dev, batch_size)
test_batches = batchify_data(X_test, y_test, batch_size)

#################################
## Model specification TODO
model = nn.Sequential(
            nn.Linear(784, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
        )
lr=0.1
momentum=0
##################################

train_model(train_batches, dev_batches, model, lr=lr, momentum=momentum)

## Evaluate the model on test data
loss, accuracy = run_epoch(test_batches, model.eval(), None)

print ("Loss on test set:"  + str(loss) + " Accuracy on test set: " + str(accuracy))
